In [2]:
import numpy as np # liniear algebra
import pandas as pd # csv I/O
import os # file operations
import h5py # handle data that doesn't fit in memory

TRAIN_DIR = r'D:\LICENTA\train_photos'
TEST_DIR = r'D:\LICENTA\test_photos'
IMG_SIZE = 299

DATA_DIR = r'D:\LICENTA\processed_data\size_{size1}x{size2}'.format(size1=IMG_SIZE, size2=IMG_SIZE)

train_labels = pd.read_csv('train.csv')

In [ ]:
%%time
# VGG16
from keras.applications.vgg16 import VGG16 # predefined CNN Model
from keras.preprocessing import image # get the array representation of the image
from keras.applications.vgg16 import preprocess_input # get the array in a format compatible with the model

# takes around 45 seconds
model = VGG16(weights='imagenet', include_top=False, pooling='max')

model.summary()

In [ ]:
%%time
# RESNET50
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input

model = ResNet50(weights='imagenet', include_top=False, pooling='max')

model.summary()

In [ ]:
%%time
# INCEPTION V3
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input

model = InceptionV3(weights='imagenet', include_top=False, pooling='max')

model.summary()

In [ ]:
%%time
# XCEPTION
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input

model = Xception(weights='imagenet', include_top=False, pooling='avg')

model.summary()

In [ ]:
%%time
# INCEPTION_RESNET_V2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input

model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='max')

model.summary()

In [4]:
MODEL_NAME = 'xception'
FEATURE_VECTOR_SIZE = 2048

In [ ]:
def extract_features(imgs_path, img_size):
    
    features = []
    
    for path in imgs_path:
        img = image.load_img(path, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features.append(model.predict(img_array).reshape(FEATURE_VECTOR_SIZE,))
    
    return np.array(features)

In [ ]:
# CREATE TRAIN DATA FILE

with h5py.File(DATA_DIR + r'\train_images_{name}_features_avg.h5'.format(name=MODEL_NAME), 'w') as f:
    img_names = f.create_dataset('photo_id', (0,), maxshape=(None,), dtype='|S54')
    feature = f.create_dataset('feature', (0, FEATURE_VECTOR_SIZE), maxshape=(None, FEATURE_VECTOR_SIZE))

In [ ]:
# TRAIN PHOTOS

import time
from datetime import datetime

train_photos = pd.read_csv('train_photo_to_biz_ids.csv')
train_images = [os.path.join(TRAIN_DIR, str(x) + '.jpg') for x in train_photos['photo_id']]

num_train = len(train_images)
print('Number of train images:', num_train, 'started at', str(datetime.now()))
batch_size = 4000
num_done = 0
t = time.time()

for i in range(0 , num_train, batch_size):
    images = train_images[i:min(i + batch_size, num_train)]
    features = extract_features(images, IMG_SIZE)
    num_done = i + features.shape[0]
    
    with h5py.File(DATA_DIR + r'\train_images_{name}_features_avg.h5'.format(name=MODEL_NAME), 'r+') as f:
        f['photo_id'].resize((num_done,))
        f['photo_id'][i:num_done] = np.array(images).astype('|S54')
        f['feature'].resize((num_done, features.shape[1]))
        f['feature'][i:num_done,:] = features
    
    if num_done % batch_size == 0 or num_done == num_train:
        print('Train images proccesed:', num_done, 'time passed: ', '{0:.1f}'.format(time.time() - t), 'sec hour:', str(datetime.now()))


In [6]:
# Check the file content

with h5py.File(DATA_DIR + r'\train_images_{name}_features_avg.h5'.format(name=MODEL_NAME),'r') as f:
    print(r'\train_images_{name}_features.h5'.format(name=MODEL_NAME))
    for key in f.keys():
        print(key, f[key].shape)

    print("\nA photo:", f['photo_id'][0])
    print("Its feature vector (first 10-dim): ", f['feature'][0][0:10], " ...")

\train_images_xception_features.h5
feature (234842, 2048)
photo_id (234842,)

A photo: b'D:\\LICENTA\\train_photos\\204149.jpg'
Its feature vector (first 10-dim):  [ 0.00866235  0.18051882  0.81814784  0.          0.32122934  0.01428686
  0.          0.03974805  0.0128702   0.        ]  ...


In [ ]:
# CREATE TEST DATA FILE

with h5py.File(DATA_DIR + r'\test_images_{name}_features_avg.h5'.format(name=MODEL_NAME), 'w') as f:
    img_names = f.create_dataset('photo_id', (0,), maxshape=(None,), dtype='|S54')
    feature = f.create_dataset('feature', (0, FEATURE_VECTOR_SIZE), maxshape=(None, FEATURE_VECTOR_SIZE))

In [ ]:
# TEST PHOTOS

import time
from datetime import datetime

test_photos = pd.read_csv('test_photo_to_biz.csv')
test_images = [os.path.join(TEST_DIR, str(x) + '.jpg') for x in test_photos['photo_id'].unique()]

num_test = len(test_images)
print('Number of test images: ', num_test)
batch_size = 4000
t = time.time()

for i in range(0, num_test, batch_size):
    images = test_images[i:min(i + batch_size, num_test)]
    features = extract_features(images, IMG_SIZE)
    num_done = i + features.shape[0]
    
    with h5py.File(DATA_DIR + r'\test_images_{name}_features_avg.h5'.format(name=MODEL_NAME), 'r+') as f:
        f['photo_id'].resize((num_done,))
        f['photo_id'][i:num_done] = np.array(images).astype('|S54')
        f['feature'].resize((num_done, features.shape[1]))
        f['feature'][i:num_done,:] = features
    
    if num_done % batch_size == 0 or num_done == num_test:
        print('Test images proccesed:', num_done, 'time passed: ', '{0:.1f}'.format(time.time() - t), 'sec hour:', str(datetime.now()))

In [5]:
# Check the file content

with h5py.File(DATA_DIR + r'\test_images_{name}_features_avg.h5'.format(name=MODEL_NAME),'r') as f:
    for key in f.keys():
        print(key, f[key].shape)

    print("\nA photo:", f['photo_id'][0])
    print("feature vector: (first 10-dim)", f['feature'][0][0:10], " ...")

feature (237152, 2048)
photo_id (237152,)

A photo: b'D:\\LICENTA\\test_photos\\317818.jpg'
feature vector: (first 10-dim) [ 0.00152443  0.          0.02481007  0.          0.          0.14680296
  0.00110827  0.01223042  0.          0.0234869 ]  ...
